In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.4 is required in this notebook
# Earlier 2.x versions will mostly work the same, but with a few bugs
import tensorflow as tf
from tensorflow import keras

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full,X_test,y_train_full,y_test = train_test_split(housing.data,
                                                           housing.target.reshape(-1,1),random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scalled = scaler.fit_transform(X_train)
X_valid_scalled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)


In [3]:
def cube(x):
    return x **3


In [4]:
# call the python function
cube(2)

8

In [5]:
# call the python function but base on tensor typedata parameter
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [6]:
# convert the python function to tensorflow function
tf_cube = tf.function(cube)
tf_cube

In [7]:
# call -> will generate the tensor type data
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [8]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

# TF Functions and concrete functions

In [9]:
concrete_function = tf_cube.get_concrete_function(tf.constant(2.0))
concrete_function.graph

In [10]:
concrete_function(tf.constant(3.0))

<tf.Tensor: shape=(), dtype=float32, numpy=27.0>

In [11]:
concrete_function is tf_cube.get_concrete_function(tf.constant(2.0))

True

In [12]:
concrete_function is tf_cube.get_concrete_function(tf.constant(10.0))

True

# Exploring function definitions and graphs

In [13]:
concrete_function.graph

In [14]:
ops = concrete_function.graph.get_operations()
ops

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [15]:
pow_op = ops[2]
list(pow_op.inputs)

[<tf.Tensor 'x:0' shape=() dtype=float32>,
 <tf.Tensor 'pow/y:0' shape=() dtype=float32>]

In [16]:
pow_op.outputs

[<tf.Tensor 'pow:0' shape=() dtype=float32>]

In [17]:
concrete_function.graph.get_operation_by_name("x")

<tf.Operation 'x' type=Placeholder>

In [18]:
concrete_function.graph.get_tensor_by_name("Identity:0")

<tf.Tensor 'Identity:0' shape=() dtype=float32>

In [19]:
concrete_function.function_def.signature

name: "__inference_cube_16"
input_arg {
  name: "x"
  type: DT_FLOAT
}
output_arg {
  name: "identity"
  type: DT_FLOAT
}

# How tf function trace python function to extract their computation graphs

In [20]:
# scracth tf.function convert from python function to tensorflow function
#used the decorator
@tf.function
def tf_cube(x):
    print("print ",x)
    return x**3

In [21]:
result = tf_cube(tf.constant([2.0]))

print  Tensor("x:0", shape=(1,), dtype=float32)


In [22]:
result

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([8.], dtype=float32)>

In [23]:
result = tf_cube(2)
result = tf_cube(3)
result = tf_cube(tf.constant([[1., 2.]])) # New shape: trace!
result = tf_cube(tf.constant([[3., 4.], [5., 6.]])) # New shape: trace!
result = tf_cube(tf.constant([[7., 8.], [9., 10.], [11., 12.]])) # New shape: trace!

print  2
print  3
print  Tensor("x:0", shape=(1, 2), dtype=float32)
print  Tensor("x:0", shape=(2, 2), dtype=float32)
print  Tensor("x:0", shape=(3, 2), dtype=float32)


its also possible to specifiy a particular input signature

In [24]:
@tf.function(input_signature=[tf.TensorSpec([None,28,28],tf.float32)])
def shrink(images):
    print("Tracing",images)
    return images[:,::2,::2] #drop half the rows and columns


In [25]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [26]:
img_batch_1=tf.random.uniform(shape=[100,28,28])
img_batch_2=tf.random.uniform(shape=[50,28,28])
preprocessed_images=shrink(img_batch_1)#traces the function
preprocessed_images = shrink(img_batch_2)#reuses the same concrete function


Tracing Tensor("images:0", shape=(None, 28, 28), dtype=float32)


In [27]:
img_batch_3 = tf.random.uniform(shape=[2,2,2])

try :
    preprocessed_images = shrink(img_batch_3) # rejects unexpected types or shapes
except ValueError as ex:
    print(ex)

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[[0.7413678  0.62854624]
  [0.01738465 0.3431449 ]]

 [[0.51063764 0.3777541 ]
  [0.07321596 0.02137029]]], shape=(2, 2, 2), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None)).


# Using Autograph to capture control flow

A "static" for loop using range()

In [28]:
@tf.function
def add_10(x):
    for i in range(10):
        x+=1
    return x

In [29]:
add_10(tf.constant(5))

<tf.Tensor: shape=(), dtype=int32, numpy=15>

In [30]:
add_10.get_concrete_function(tf.constant(5)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'add/y' type=Const>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'add_1/y' type=Const>,
 <tf.Operation 'add_1' type=AddV2>,
 <tf.Operation 'add_2/y' type=Const>,
 <tf.Operation 'add_2' type=AddV2>,
 <tf.Operation 'add_3/y' type=Const>,
 <tf.Operation 'add_3' type=AddV2>,
 <tf.Operation 'add_4/y' type=Const>,
 <tf.Operation 'add_4' type=AddV2>,
 <tf.Operation 'add_5/y' type=Const>,
 <tf.Operation 'add_5' type=AddV2>,
 <tf.Operation 'add_6/y' type=Const>,
 <tf.Operation 'add_6' type=AddV2>,
 <tf.Operation 'add_7/y' type=Const>,
 <tf.Operation 'add_7' type=AddV2>,
 <tf.Operation 'add_8/y' type=Const>,
 <tf.Operation 'add_8' type=AddV2>,
 <tf.Operation 'add_9/y' type=Const>,
 <tf.Operation 'add_9' type=AddV2>,
 <tf.Operation 'Identity' type=Identity>]

A "dynamic" loop using tf.while_loop()

In [31]:
@tf.function
def add_10(x):
    condition = lambda i,x : tf.less(i,10)
    body = lambda i,x: (tf.add(i,1), tf.add(x,1))
    final_i,final_x = tf.while_loop(condition,body,[tf.constant(0),x])
    return final_x

In [32]:
add_10(tf.constant(5))

<tf.Tensor: shape=(), dtype=int32, numpy=15>

In [33]:
add_10.get_concrete_function(tf.constant(5)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'while/maximum_iterations' type=Const>,
 <tf.Operation 'while/loop_counter' type=Const>,
 <tf.Operation 'while' type=StatelessWhile>,
 <tf.Operation 'Identity' type=Identity>]